In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The data

In [3]:
# train_df = pd.read_csv('../data/25_jan/train_set_basic.csv')
train_df = pd.read_csv('../data/missingness/0/training_set.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [4]:
train_df.cutaneous_lupus.value_counts()

0    44200
3     3120
1     2246
2      834
Name: cutaneous_lupus, dtype: int64

In [5]:
train_df[(train_df.ana==0) & (train_df.label==1)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label


In [6]:
train_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [7]:
len(train_df)

50400

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [9]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No lupus',
 'Lupus',
 'Inconclusive diagnosis',
 'ana',
 'fever',
 'leukopenia',
 'thrombocytopenia',
 'auto_immune_hemolysis',
 'delirium',
 'psychosis',
 'seizure',
 'non_scarring_alopecia',
 'oral_ulcers',
 'cutaneous_lupus',
 'pleural_effusion',
 'pericardial_effusion',
 'acute_pericarditis',
 'joint_involvement',
 'proteinuria',
 'anti_cardioliphin_antibodies',
 'anti_β2gp1_antibodies',
 'lupus_anti_coagulant',
 'low_c3',
 'low_c4',
 'anti_dsdna_antibody',
 'anti_smith_antibody']

In [10]:
# steps = 11279933
# dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../models/24_feb/dqn_basic_{steps}')

In [11]:
#0.92 at 11,279,933 timesteps and 1900000 episodes also at 36,144,785 timestepsc(7800000 episodes)

In [12]:
# for steps in [int(5e4), int(1e5), int(3e5), int(5e5), int(8e5), int(1e6), int(2e6)]:    
# # for steps in [int(1.1e5)]:  
#     dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../models/26_jan/dqn_basic_{steps}')

#### The Model

In [14]:
from stable_baselines import ACER
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines.common.callbacks import CheckpointCallback
from modules.env import LupusEnv

In [15]:
training_env = LupusEnv(X_train, y_train)
training_env = DummyVecEnv([lambda: training_env])
# training_env = VecNormalize(training_env, norm_obs=True, norm_reward=False, clip_obs=10.)

model = ACER('MlpPolicy', training_env, verbose=1, seed = constants.SEED)
checkpoint_callback = CheckpointCallback(save_freq=100000, save_path='../models/logs/acer', name_prefix='acer_basic_')

model.learn(total_timesteps=70000000, log_interval=100000, callback=checkpoint_callback)

# Save the trained DQN agent
model.save('../models/22_mar/acer_lupus_diagnosis')
training_env.close()

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
-----------------------------------
| avg_norm_adj        | 0.151     |
| avg_norm_g          | 21.6      |
| avg_norm_grads_f    | 21.5      |
| avg_norm_k          | 5.1       |
| avg_norm_k_dot_g    | 21.6      |
| entropy             | 68.4      |
| explained_variance  | -8.93e-05 |
| fps                 | 0         |
| loss                | 1.16      |
| loss_bc             | -0        |
| loss_f              | 1.64      |
| loss_policy         | 1.64      |
| loss_q              | 0.399     |
| mean_episode_length | 2.83      |
| mean_episode_reward | 0.606     |
| norm_grads          | 1.01      |
| norm_grads_policy   | 0.88      |
| norm_grads_q        | 0.505     |
| total_timesteps     | 20        |
-----------------------------------
----------------------------------
| avg_norm_adj        | 0.00

----------------------------------
| avg_norm_adj        | 0.0437   |
| avg_norm_g          | 0.799    |
| avg_norm_grads_f    | 0.76     |
| avg_norm_k          | 1.8      |
| avg_norm_k_dot_g    | 0.8      |
| entropy             | 1.04     |
| explained_variance  | 0.0817   |
| fps                 | 1812     |
| loss                | 0.203    |
| loss_bc             | -0       |
| loss_f              | -0.00321 |
| loss_policy         | -0.00321 |
| loss_q              | 0.434    |
| mean_episode_length | 2        |
| mean_episode_reward | 0.417    |
| norm_grads          | 0.206    |
| norm_grads_policy   | 0.0433   |
| norm_grads_q        | 0.201    |
| total_timesteps     | 20000020 |
----------------------------------
----------------------------------
| avg_norm_adj        | 0.0188   |
| avg_norm_g          | 0.558    |
| avg_norm_grads_f    | 0.541    |
| avg_norm_k          | 1.55     |
| avg_norm_k_dot_g    | 0.559    |
| entropy             | 0.677    |
| explained_variance

----------------------------------
| avg_norm_adj        | 0.0587   |
| avg_norm_g          | 0.894    |
| avg_norm_grads_f    | 0.841    |
| avg_norm_k          | 1.58     |
| avg_norm_k_dot_g    | 0.853    |
| entropy             | 5.61     |
| explained_variance  | 0.0412   |
| fps                 | 1991     |
| loss                | 0.154    |
| loss_bc             | -0       |
| loss_f              | 0.0575   |
| loss_policy         | 0.0575   |
| loss_q              | 0.305    |
| mean_episode_length | 2.15     |
| mean_episode_reward | 0.462    |
| norm_grads          | 0.359    |
| norm_grads_policy   | 0.344    |
| norm_grads_q        | 0.105    |
| total_timesteps     | 44000020 |
----------------------------------
----------------------------------
| avg_norm_adj        | 0.11     |
| avg_norm_g          | 1.23     |
| avg_norm_grads_f    | 1.15     |
| avg_norm_k          | 1.81     |
| avg_norm_k_dot_g    | 1.19     |
| entropy             | 5.54     |
| explained_variance

----------------------------------
| avg_norm_adj        | 0.0277   |
| avg_norm_g          | 0.677    |
| avg_norm_grads_f    | 0.653    |
| avg_norm_k          | 1.85     |
| avg_norm_k_dot_g    | 0.676    |
| entropy             | 2.86     |
| explained_variance  | 0.0276   |
| fps                 | 2271     |
| loss                | 0.127    |
| loss_bc             | -0       |
| loss_f              | 0.00128  |
| loss_policy         | 0.00128  |
| loss_q              | 0.309    |
| mean_episode_length | 2.05     |
| mean_episode_reward | 0.465    |
| norm_grads          | 0.247    |
| norm_grads_policy   | 0.0261   |
| norm_grads_q        | 0.246    |
| total_timesteps     | 68000020 |
----------------------------------
